In [392]:

import xlwings as xw
import os
import pandas as pd
import numpy as np
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [405]:

#-- Batch 5A defined
batch5a = 'FSC-JAD-01-TEC-61-A1 MV FMECA-Batch 5 LBHD - AN.xlsx' 
pathBatch5a = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets'
shtBatch5a = '5- 4S WBT FMECA'
cellRangeBatch5a = 'A5:BP514'


Assigne the above variables to the ones used in this workbook

In [406]:
nameFile = batch5a
path = pathBatch5a
sheet = shtBatch5a
cellRange = cellRangeBatch5a

In [407]:
file_path = os.path.join(path,nameFile)
book = xw.Book(file_path)
sht = book.sheets[sheet]
rng = sht.range(cellRange)
df = rng.options(pd.DataFrame, index=False, header=True).value

In [408]:
df.head(1)


,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,Coating condition
0,1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,None,None,None,None,Local Generalised corrosion on the longitudina...,None,Poor


### Stiffener problem

The stiffeners listed in the Vertech reports have some inconsistencies. <br>


Some appear as LL06-07 <br>

    In this case it will be replaced with the "nearest longitudinal stiffener". 
    It should be either one the above.
    
Some appear as HG1, HG2 etc. <br>

    HG1, HG2, HG3 are also the same as LL09, LL16, and LL22 respectively.
    They will be replaced with the stiffener names.

This will create a new column. <br>

The column should be coppied and replaced with the "stiffener" column in the FMECA sheet.


In [409]:
def stiffenerCorrection(row):
    stiffener = row['Stiffener(s)']
    nearest_lng = row['Nearest longitudinal member (SS,ObLBHD)']
    if stiffener == 'HG1':
        return 'LL09'
    elif stiffener == 'HG2':
        return 'LL16'
    elif stiffener == 'HG3':
        return 'LL22'
    elif len(stiffener.split('-')) == 2:
        return nearest_lng
    else:
        return stiffener

In [410]:
df['new_stiffener'] = df.apply(stiffenerCorrection,axis=1)

In [411]:
df.head(1)

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,Coating condition,new_stiffener
0,1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,None,None,None,None,Local Generalised corrosion on the longitudina...,None,Poor,LL15


In [412]:
abt_lbhd = {"LL00":14,
"LL01":14,
"LL02":14,
"LL03":14,
"LL04":11.5,
"LL05":11.5,
"LL06":11.5,
"LL07":11.5,
"LL08":12,
"LL09":12,
"LL10":12,
"LL11":12,
"LL12":13,
"LL13":13,
"LL14":13,
"LL15":13,
"LL16":14.5,
"LL17":14.5,
"LL18":14.5,
"LL19":14.5,
"LL20":15,
"LL21":15,
"LL22":15,
"LL23":16,
"LL24":16,
"LL25":16,
"LL26":19.5,
"LL27":19.5,
"UD":20.5,
"BP":18.0
}

In [413]:
abt_web = {"LL00":10,
"LL01":10,
"LL02":10,
"LL03":10,
"LL04":10,
"LL05":10,
"LL06":10,
"LL07":10,
"LL08":10,
"LL09":10,
"LL10":10,
"LL11":10,
"LL12":10,
"LL13":10,
"LL14":10,
"LL15":10,
"LL16":10,
"LL17":10,
"LL18":10.5,
"LL19":10.5,
"LL20":10.5,
"LL21":10.5,
"LL22":10.5,
"LL23":10.5,
"LL24":10.5,
"LL25":10.5,
"LL26":10.5,
"LL27":10.5,
"UD":12.0,
"BP":13.5
}


In [414]:
abt_flange = {"LL00":15.5,
"LL01":15.5,
"LL02":17,
"LL03":17,
"LL04":18.5,
"LL05":18.5,
"LL06":17,
"LL07":17,
"LL08":20,
"LL09":20,
"LL10":20,
"LL11":20,
"LL12":20,
"LL13":20,
"LL14":22,
"LL15":22,
"LL16":19.5,
"LL17":19.5,
"LL18":18,
"LL19":18,
"LL20":20,
"LL21":21,
"LL22":23,
"LL23":23,
"LL24":19.5,
"LL25":20.5,
"LL26":24.5,
"LL27":14,
"UD":17,
"BP":25
}

In [415]:
def abt_check(row):
    
    if row['Local Structure'] == 'Plate':
        if row['Stiffener(s)'] in abt_lbhd:
            if abt_lbhd[row['Stiffener(s)']] == row["As Built Thickness\n(mm)"]:
                return 'abt correct'
            else:
                return 'check abt'
        else:
            return 'find abt from drawings'
    elif row['Local Structure'] == "Long'l stiff'r":
        if row['Detail Structure'] == 'Web':
            if row['Stiffener(s)'] in abt_web:
                if abt_web[row['Stiffener(s)']] == row["As Built Thickness\n(mm)"]:
                    return 'abt correct'
                else:
                    return 'check abt'
            else:
                return 'find abt from drawings'
        elif row['Detail Structure'] == 'Flange':
            if row['Stiffener(s)'] in abt_flange:
                if abt_flange[row['Stiffener(s)']] == row["As Built Thickness\n(mm)"]:
                    return 'abt correct'
                else:
                    return 'check abt'
            else:
                return 'find abt from drawings'

        else:
            return 'not a web/flange'
    else:
        return 'not a plate/stiffener'
        

In [416]:
df['abtCheck'] = df.apply(abt_check, axis=1)

In [417]:
def correct_abt(row):
    vertech_abt = row["As Built Thickness\n(mm)"]
    if row['Local Structure'] == 'Plate':
        if row['abtCheck'] == 'check abt':
            return abt_lbhd[row['Stiffener(s)']]
        else:
            return vertech_abt
    elif row['Local Structure'] == "Long'l stiff'r":
        if row['Detail Structure'] == 'Web':
            if row['abtCheck'] == 'check abt':
                return abt_web[row['Stiffener(s)']]
            else:
                return vertech_abt
        elif row['Detail Structure'] == 'Flange':
            if row['abtCheck'] == 'check abt':
                return abt_flange[row['Stiffener(s)']]
            else:
                return vertech_abt
    else:
        return vertech_abt
        

In [418]:

df['correctAbt'] = df.apply(correct_abt, axis=1)

In [419]:
correct_abts = df[['Anomaly ID','Stiffener(s)','As Built Thickness\n(mm)','Local Structure','Detail Structure','abtCheck','correctAbt','new_stiffener']]

In [420]:
correct_abts.shape

(509, 8)

#### As-built thickness corrected table

In order to view the table uncommate the line below.

In [421]:
#xw.view(correct_abts)

In [422]:
def finalAnomalyDescription(row):
    if row['Final Anomaly description'] == 'Pit within generalised corrosion':
        return 'Pit within generalised corrosion'
    else:
        if row['Anomaly Nature'] == 'Corrosion':
            return 'Generalised corrosion'
        elif row['Anomaly Nature'] == 'Pitting':
            return 'Pit'
        else:
            return ''

In [423]:
df['final_anomlay_description'] = df.apply(finalAnomalyDescription,axis=1)

In [424]:
def locationStructure(row):
    if row['Local Structure'] == 'Plate':
        return 'Tank boundary plating'
    elif row['Local Structure'] == "Long'l stiff'r":
        if row['Detail Structure'] == 'Web':
            return 'Stiffener web'
        elif row['Detail Structure'] == 'Flange':
            return 'Stiffener flange middle 1/3'
    else:
        return ''

In [425]:
df['location_formulated'] = df.apply(locationStructure,axis=1)

In [426]:
def detailStatus(row):
    remaining_t = row['Average UTM Reading (mm)']
    loss = row['(mm)']
    renewable_loss = row["As Built Thickness\n(mm)"] - row["Renewal Thickness (mm)"]
    substantial_loss = row["As Built Thickness\n(mm)"] - row["Substantial Corer Thickness (mm)"]
    flag = row['FLAG']

    if row['Final Anomaly description'] == 'Pit':
        if row['Location'] == 'Tank boundary plating':
            if remaining_t > 10.0:
                if flag == "R":
                    return 'More than 10mm remaining and greater than renewable loss'
                else:
                    return "Substantial or below substantial loss"
            elif 6.0 <= remaining_t and  remaining_t <= 10.0:
                if flag == "R":
                    return 'Between 6 - 10mm remaining'
                else:
                    return "Substantial or below substantial loss"
            elif remaining_t < 6.0:
                return 'Less than 6mm remaining'
            else:
                return 'investigate'
        if row['Location'] == 'Stiffener web':
            if loss > renewable_loss:
                if remaining_t < 6.0:
                    return 'Any loss greater than renewable (<6mm)'
                else:
                    return 'Any loss greater than renewable'
            else:
                return "Substantial or below substantial loss"
        if row['Location'] == 'Stiffener flange middle 1/3':
            if loss > renewable_loss:
                if remaining_t < 6.0:
                    return 'Any loss greater than renewable (<6mm)'
                else:
                    return 'Any loss greater than renewable'
            else:
                return "Substantial or below substantial loss"
        if row['Location'] == 'Stiffener flange outside of middle 1/3':
            if remaining_t >= 6.0:
                if flag == "R":
                    return 'More than 6mm remaining and greater than renewable loss'
                else:
                    return "Substantial or below substantial loss"
            elif remaining_t < 6.0:
                return 'Less than 6mm remaining'
    if row['Final Anomaly description'] == 'Generalised corrosion':
        if remaining_t < 6.0:
            return 'Less than 6mm remaining'
        else:
            return "Greater than 6mm remaining"
    if row['Final Anomaly description'] == 'Pit within generalised corrosion':
        if remaining_t < 6.0:
            return 'Less than 6mm remaining'
        else:
            return "Greater than 6mm remaining"


    

In [427]:
def repairMethod(row):
    detail = row['Detail']
    if row['Final Anomaly description'] == 'Pit':
        if row['Location'] == 'Tank boundary plating':
            if detail == 'More than 10mm remaining and greater than renewable loss':
                return 'investigate'
            elif detail == 'Between 6 - 10mm remaining':
                return 'Arrest and monitor CVI'
            elif detail == 'Less than 6mm remaining':
                return 'Permanent repair'
            else:
                return 'investigate'
        if row['Location'] == 'Stiffener web':
            if detail == 'Any loss greater than renewable':
                return 'investigate coating condition'
            else:
                return 'investigate coating condition'
        if row['Location'] == 'Stiffener flange middle 1/3':
            if detail == 'Any loss greater than renewable':
                return 'Arrest and monitor CVI'
            else:
                return 'investigate'
        if row['Location'] == 'Stiffener flange outside of middle 1/3':
            if detail == 'More than 6mm remaining and greater than renewable loss':
                return 'A&M CVI or investigate'
            elif detail == 'Less than 6mm remaining':
                return 'investigate'
    if row['Final Anomaly description'] == 'Generalised corrosion':
        if detail == 'More than 10mm remaining and greater than renewable loss':
            return 'investigate'
        elif detail == 'Between 6 - 10mm remaining':
            return 'Arrest and monitor CVI'
        elif detail == 'Less than 6mm remaining':
            return 'Permanent repair'
        else:
            return 'investigate'
    if row['Final Anomaly description'] == 'Pit within generalised corrosion':
        if detail == 'More than 10mm remaining and greater than renewable loss':
            return 'investigate'
        elif detail == 'Between 6 - 10mm remaining':
            return 'Arrest and monitor CVI'
        elif detail == 'Less than 6mm remaining':
            return 'Permanent repair'
        else:
            return 'investigate'

In [428]:
def RepairMethodR1(row):
    detail = row['Detail']
    coating = row['Coating condition']
    anomaly_type = row['Final Anomaly description']
    location = row['Location']

    if coating != "":
        if anomaly_type == 'Pit':
            if location == 'Tank boundary plating':
                if detail == 'More than 10mm remaining and greater than renewable loss' or detail == "Substantial or below substantial loss":
                    if coating == 'Fair' or coating == 'Poor':
                        return "Arrest and monitor CVI"
                    else:
                        return "Monitor only, no coating"
                elif detail == 'Between 6 - 10mm remaining':
                    return 'Arrest and monitor CVI'
                elif detail == 'Less than 6mm remaining':
                    return 'Permanent repair'
                elif detail == "Substantial or below substantial loss":
                    if coating == 'Fair' or coating == 'Poor':
                        return "Arrest and monitor CVI"
                    else:
                        return "Monitor only, no coating"
                else:
                    return 'Pit. not above 10 or below 10?'

            elif location == 'Stiffener web':
                if detail == 'Any loss greater than renewable':
                    return "Arrest and monitor CVI"
                elif detail == 'Any loss greater than renewable (<6mm)':
                    return 'Permanent repair'
                else:
                    if coating == 'Fair' or coating == 'Poor':
                        return "Arrest and monitor CVI"
                    else:
                        return "Monitor only, no coating"
            elif location == 'Stiffener flange middle 1/3':
                if detail == 'Any loss greater than renewable':
                    return "Arrest and monitor CVI"
                elif detail == 'Any loss greater than renewable (<6mm)':
                    return 'Permanent repair'
                else:
                    if coating == 'Fair' or coating == 'Poor':
                        return "Arrest and monitor CVI"
                    else:
                        return "Monitor only, no coating"
            elif location == 'Stiffener flange outside of middle 1/3':
                if detail == 'More than 6mm remaining and greater than renewable loss':
                    return 'Permanent repair'
                elif detail == 'Less than 6mm remaining':
                    return 'Permanent repair'
        if anomaly_type == 'Generalised corrosion':
            
            if detail == 'Less than 6mm remaining':
                return 'Structural repair'
            elif detail == "Greater than 6mm remaining":
                if coating == 'Fair' or coating == 'Poor':
                    return "Arrest and monitor CVI"
                else:
                    return "Monitor only (not part of standard repair)"
            else:
                return 'Corr. not above 10, or below 10?'
        if anomaly_type == 'Pit within generalised corrosion':
            if detail == 'Less than 6mm remaining':
                return 'Structural repair'
            elif detail == "Greater than 6mm remaining":
                if coating == 'Fair' or coating == 'Poor':
                    return "Arrest and monitor CVI"
                else:
                    return "Monitor only (not part of standard repair)"
            else:
                return 'Corr. not above 10, or below 10?'
    else:
        return 'define coating condition'


In [429]:
df['detail_formulated'] = df.apply(detailStatus,axis=1)

TypeError: unsupported operand type(s) for -: 'str' and 'float'

In [430]:
df.head(1)

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,Coating condition,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated
0,1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,None,None,None,None,Local Generalised corrosion on the longitudina...,None,Poor,LL15,abt correct,13.0,Generalised corrosion,Tank boundary plating


In [431]:
dm = 'decisionMatrix.xlsx' 
pathANworking = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets/AN_working'
decisionMatrixPath = os.path.join(pathANworking,dm)

In [432]:
dimLevels = pd.read_excel(decisionMatrixPath,sheet_name='diminutionLevels')

In [433]:
dm = pd.read_excel(decisionMatrixPath,sheet_name='decisionMatrix')

In [434]:
dimLevels

,Primary structure,Local structure,Diminution level
0,Long'l bhd,Plate,0.20
1,Long'l bhd,Long'l stiff'r,0.25


In [435]:
dm

,Anomaly type,Location,Detail,Repair,Inspection interval
0,Pit,Tank boundary plating,More than 10mm remaining and greater than rene...,Permanent repair,Scheduled tank inspection
1,Pit,Tank boundary plating,More than 10mm remaining and greater than rene...,Temporary repair,1 year λ
2,Pit,Tank boundary plating,More than 10mm remaining and greater than rene...,Arrest and monitor CVI,2.5 years
3,Pit,Tank boundary plating,More than 10mm remaining and greater than rene...,"Monitor only, no coating",1 year
4,Pit,Tank boundary plating,Between 6 - 10mm remaining,Permanent repair,Scheduled tank inspection
5,Pit,Tank boundary plating,Between 6 - 10mm remaining,Temporary repair,1 year λ
6,Pit,Tank boundary plating,Between 6 - 10mm remaining,Arrest and monitor CVI,1 year λ
7,Pit,Tank boundary plating,Less than 6mm remaining,Permanent repair,Scheduled tank inspection
8,Pit,Tank boundary plating,Less than 6mm remaining,Temporary repair,1 year
9,Pit,Tank boundary plating,Substantial or below substantial loss,Arrest and monitor CVI,1 year λ


In [436]:
def inspectionInterval(row):
    anomaly_type = row['Final Anomaly description']
    location = row['Location']
    detail = row['Detail']
    repair = row['Repair']

    if anomaly_type == 'Pit':
        a = dm[dm['Anomaly type']==anomaly_type]
        b = a[a['Location']==location]
        c = b[b['Detail']==detail]
        d = c[c['Repair'] == repair]
        try:
            interval = d.iloc[0,-1]
        except:
            interval = 'error'
    else:
        b = dm[dm['Anomaly type']==anomaly_type]
        c = b[b['Detail']==detail]
        d = c[c['Repair'] == repair]
        try:
            interval = d.iloc[0,-1]
        except:
            interval = 'error'

    return interval
    
    



In [437]:
df['inspInt_formulate'] = df.apply(inspectionInterval,axis=1)

In [438]:
def allowablePercentage(row):

    primary_structure = row["Primary Structure"]
    local_structure = row["Local Structure"]

    a = dimLevels[dimLevels['Primary structure']==primary_structure]
    b = a[a['Local structure']==local_structure]
    try:
        allowable = b.iloc[0,-1]
    except:
        allowable = float(0)

    return allowable


In [439]:
def allowablechecks(row):
    dimLevelVertech = float(row["Max Allowable Diminution (%)"])
    dimLevelFSC = float(row["allowables"])
    diff = np.abs(dimLevelVertech-dimLevelFSC)
    if diff == 0:
        return 'correct dim level'
    else:
        return 'check dim level'



In [440]:
df['allowables'] = df.apply(allowablePercentage,axis=1)

In [441]:
df['allowableCheck'] = df.apply(allowablechecks,axis=1)

In [442]:
df.head(3)

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,Coating condition,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,inspInt_formulate,allowables,allowableCheck
0,1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.7900,3.7,6.2100,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,None,None,None,None,Local Generalised corrosion on the longitudina...,None,Poor,LL15,abt correct,13.0,Generalised corrosion,Tank boundary plating,error,0.2,correct dim level
1,2.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-193,AW,49-50,LL21,LBHD,4000.0,100.0,FR.49,50.0,LL21,0.0,AH,48.7,15.0,0.2,12.0,12.75,6.9,5.8,4.1,8.5,10.8,11.8,12.1,12.1,NaN,NaN,9.0125,4.1,5.9875,0.399167,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,5.0,100.0,0.399167,Local,Med,Med,Y,Generalised corrosion,None,None,None,None,Local Generalised corrosion on the longitudina...,None,Poor,LL21,abt correct,15.0,Generalised corrosion,Tank boundary plating,error,0.2,correct dim level
2,3.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-194,AW,50-51,LL11,LBHD,15.0,15.0,FR.51,N/A,LL11,N/A,AH,0.0,12.0,0.2,9.6,10.20,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.5000,5.5,6.5000,0.541667,R,Long'l bhd,Panel,Plate,None,Pitting,Long'l bhdPanelPlatePitting,8.0,7.0,8.0,448.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,8.0,3.0,4.0,96.0,NaN,0.0,0.541667,Local,Med,Med,None,Pit,Tank boundary plating,Less than 6mm remaining,Permanent repair,Scheduled tank inspection,Local Pit found on the longitudinal bulkhead p...,Structural repair (Pad weld),Good,LL11,abt correct,12.0,Pit,Tank boundary plating,Scheduled tank inspection,0.2,correct dim level


In [443]:
df['repair_formulate'] = df.apply(RepairMethodR1,axis=1)

In [444]:
df.head(1)

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,Coating condition,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,inspInt_formulate,allowables,allowableCheck,repair_formulate
0,1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,None,None,None,None,Local Generalised corrosion on the longitudina...,None,Poor,LL15,abt correct,13.0,Generalised corrosion,Tank boundary plating,error,0.2,correct dim level,"Corr. not above 10, or below 10?"


In [445]:
summary_df_cols = ['Report reference', 'Anomaly ID','Stiffener(s)','Nearest longitudinal member (SS,ObLBHD)',
       'As Built Thickness\n(mm)', 
       'Renewal Thickness (mm)', 'Substantial Corer Thickness (mm)',
       'Average UTM Reading (mm)', 'Minimum \nUTT \nReading ', '(mm)', '(%)',
       'FLAG','correctAbt',
       'final_anomlay_description',
       'location_formulated', 'detail_formulated', 'repair_formulate']

In [446]:
#xw.view(df)

In [447]:
df.head(1)

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,Coating condition,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,inspInt_formulate,allowables,allowableCheck,repair_formulate
0,1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,None,None,None,None,Local Generalised corrosion on the longitudina...,None,Poor,LL15,abt correct,13.0,Generalised corrosion,Tank boundary plating,error,0.2,correct dim level,"Corr. not above 10, or below 10?"


In [448]:
summary_allowable = ['Anomaly ID','Stiffener(s)','Max Allowable Diminution (%)','Primary Structure','Local Structure','allowables','allowableCheck']

In [449]:
#xw.view(df[summary_allowable])

### Function for notes


In [450]:
def notes(row):
    if row['Final Anomaly description'] != 'Pit within generalised corrosion':
        anomaly_type = 'Local '+ str(row['Final Anomaly description'])
    else:
        anomaly_type = row['Final Anomaly description']
    
    if row['Primary Structure'] == "Long'l bhd":
        structural_component = 'longitudinal bulkhead'

    if row["Local Structure"] == "Plate":
        local_component = 'plate'
    elif row["Local Structure"] == "Long'l stiff'r":
        local_component = "stiffener"
    else:
        local_component = ''

    if row["Detail Structure"] is None:
        detail_structure = ''
    else:
        detail_structure = row["Detail Structure"]
    coating_condition = row["Coating condition"].lower()

    if row["FLAG"] == 'R':
        flag = 'has reached renewal'
        calcs = 'calcs'
    elif row["FLAG"] == 'SC':
        flag = 'has reached substantial (not renewal)'
        calcs = 'no calcs'
    else:
        flag = 'is not at a critical'
        calcs = 'no calcs'
    remaining_t = str(row["Average UTM Reading (mm)"])

    
    if row['Final Anomaly description'] == 'Pit' and row["Average UTM Reading (mm)"] < 6.0:
         notes_text = f'{anomaly_type} found on the {structural_component} {local_component} with {coating_condition} coating. Remaining thickness {remaining_t}mm.'
    else:
        notes_text = f'{anomaly_type} on the {structural_component} {local_component} {detail_structure}. Average UT {flag} limit therefor {calcs} required. Coating condition is {coating_condition}.'
    notes_text = notes_text.replace(' .','.')
    return notes_text

In [451]:
df['notes_formulated'] = df.apply(notes,axis=1)

In [ ]:
xw.view(df)

In [452]:
'Good'.lower()

'good'

### Table of stiffeners that require calculations

In [453]:
#-- import the dataset containing stiffener dimensions
stiff_dim = pd.read_excel(decisionMatrixPath,sheet_name='Stiffeners')

In [454]:
stiff_dim['Stiffener name'].unique()

array(['LL00', 'LL01', 'LL02', 'LL03', 'LL04', 'LL05', 'LL06', 'LL07',
       'LL08', 'LL09', 'LL10', 'LL11', 'LL12', 'LL13', 'LL14', 'LL15',
       'LL16', 'LL17', 'LL18', 'LL19', 'LL20', 'LL21', 'LL22', 'LL23',
       'LL24', 'LL25', 'LL26', 'LL27', 'BP', 'UD'], dtype=object)

In [455]:
stiff_dim[stiff_dim['Stiffener name']=='LL19'].iloc[0,1]

'500x10.5 + 150x18 L2'

In [471]:
#-- Filtering a dataset that requires calculations
req_calc = df[~(df['Final Anomaly description']=='Pit') & (df['FLAG']=='R')]

In [476]:
req_calc[req_calc["Stiffener(s)"].isnull()]

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,Coating condition,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,inspInt_formulate,allowables,allowableCheck,repair_formulate,notes_formulated,stiffener_dim


In [473]:
set(set(req_calc['Stiffener(s)'])).difference(stiff_dim['Stiffener name'])

set()

In [474]:
def stiff_dimensions(row,col_number):
    stiffener = row['Stiffener(s)']
    filter_dim = stiff_dim[stiff_dim['Stiffener name']==str(stiffener)].iloc[0,col_number]
    return filter_dim

In [505]:
def concatStiffners(row):
    return row['Stiffener(s)'] + "_" + row['Anomaly ID']

In [475]:
req_calc['stiffener_dim'] = req_calc.apply(stiff_dimensions,args = (1,),axis=1)


C:\Users\nisha\AppData\Local\Temp\ipykernel_17340\3792857481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_calc['stiffener_dim'] = req_calc.apply(stiff_dimensions,args = (1,),axis=1)


In [478]:
req_calc['h'] = req_calc.apply(stiff_dimensions,args = (2,),axis=1)


C:\Users\nisha\AppData\Local\Temp\ipykernel_17340\1320414672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_calc['h'] = req_calc.apply(stiff_dimensions,args = (2,),axis=1)


In [479]:
req_calc['grade'] = req_calc.apply(stiff_dimensions,args = (3,),axis=1)
req_calc['ps'] = req_calc.apply(stiff_dimensions,args = (4,),axis=1)


C:\Users\nisha\AppData\Local\Temp\ipykernel_17340\1251418026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_calc['grade'] = req_calc.apply(stiff_dimensions,args = (3,),axis=1)
C:\Users\nisha\AppData\Local\Temp\ipykernel_17340\1251418026.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_calc['ps'] = req_calc.apply(stiff_dimensions,args = (4,),axis=1)


In [506]:
req_calc['stiff_anom_combo'] = req_calc.apply(concatStiffners,axis=1)

C:\Users\nisha\AppData\Local\Temp\ipykernel_17340\1422889332.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_calc['stiff_anom_combo'] = req_calc.apply(concatStiffners,axis=1)


In [508]:
req_calc.head(1)

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,Coating condition,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,inspInt_formulate,allowables,allowableCheck,repair_formulate,notes_formulated,stiffener_dim,h,grade,ps,stiff_anom_combo
0,1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,None,None,None,None,Local Generalised corrosion on the longitudina...,None,Poor,LL15,abt correct,13.0,Generalised corrosion,Tank boundary plating,error,0.2,correct dim level,"Corr. not above 10, or below 10?",Local Generalised corrosion on the longitudina...,400x10 + 150x22 L2,11.13,AH36,Sec,LL15_4SWBT-LBHD- CR-192


In [509]:
req_calc_cols = ["Stiffener(s)","stiff_anom_combo","stiffener_dim","h","grade","ps","Average UTM Reading (mm)"]

In [510]:
reqCalc = req_calc[req_calc_cols]

In [511]:
reqCalc.sort_values(by='Stiffener(s)',ascending=True,inplace=True)

C:\Users\nisha\AppData\Local\Temp\ipykernel_17340\2518793318.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reqCalc.sort_values(by='Stiffener(s)',ascending=True,inplace=True)


In [512]:
xw.view(reqCalc)